### Today:
* Reccurent Neural Networks
    * Vanilla
    * LSTM

### Resources:
* RNN: https://www.tensorflow.org/tutorials/text_classification_with_tf_hub
* RNN: http://colah.github.io/posts/2015-08-Understanding-LSTMs/
* RNN: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [3]:
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
# Import MNIST data
mnist = input_data.read_data_sets("../../data/05_kNN/MNIST_data/", one_hot=True)

Extracting ../../data/05_kNN/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../data/05_kNN/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../data/05_kNN/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../data/05_kNN/MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
# Training Parameters
learning_rate = 0.001
training_steps = 5000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

In [8]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
W = tf.Variable(tf.random_normal([num_hidden, num_classes]))
b = tf.Variable(tf.random_normal([num_classes]))

In [9]:
# Prepare data shape to match `rnn` function requirements
# Current data input shape: (batch_size, timesteps, n_input)
# Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

# Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
x = tf.unstack(X, timesteps, 1)

# Define a lstm cell with tensorflow
lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

# Get lstm cell output
outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

# Linear activation, using rnn inner loop last output
logits = tf.matmul(outputs[-1], W) + b

prediction = tf.nn.softmax(logits)

# Loss
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(tf.global_variables_initializer())

    for step in range(1, training_steps + 1):
        
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={X: batch_x, Y: batch_y})
        
        if step % display_step == 0 or step == 1:
            
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            
            print('Epoch: %04d, cost: %4.10f, accuracy: %4.10f' % (step, loss, acc))


    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    
    print("Test acc: %4.10f" % sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Epoch: 0001, cost: 2.6241097450, accuracy: 0.0781250000
Epoch: 0200, cost: 0.2421981245, accuracy: 0.9218750000
Epoch: 0400, cost: 0.1481991112, accuracy: 0.9765625000
Epoch: 0600, cost: 0.0722186342, accuracy: 0.9765625000
Epoch: 0800, cost: 0.1444121450, accuracy: 0.9687500000
Epoch: 1000, cost: 0.0344634801, accuracy: 1.0000000000
Epoch: 1200, cost: 0.0120857675, accuracy: 1.0000000000
Epoch: 1400, cost: 0.0648216605, accuracy: 0.9843750000
Epoch: 1600, cost: 0.2094063461, accuracy: 0.9531250000
Epoch: 1800, cost: 0.0221068915, accuracy: 0.9921875000
Epoch: 2000, cost: 0.0790174529, accuracy: 0.9765625000
Epoch: 2200, cost: 0.0250653476, accuracy: 0.9921875000
Epoch: 2400, cost: 0.0448347293, accuracy: 0.9921875000
Epoch: 2600, cost: 0.0134690702, accuracy: 1.0000000000
Epoch: 2800, cost: 0.0048909411, accuracy: 1.0000000000
Epoch: 3000, cost: 0.0152390515, accuracy: 0.9921875000
Epoch: 3200, cost: 0.0143390931, accuracy: 0.9921875000
Epoch: 3400, cost: 0.0186426397, accuracy: 0.992